<a href="https://colab.research.google.com/github/Uzair-DeVops/Quater-2/blob/main/Assignment_05_Exploring_Gemini_2_0_Video_and_Audio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet google-genai


In [2]:
from google.colab import userdata
import os
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY_3')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [33]:
from IPython.display import display, Audio ,Markdown


In [3]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

#1. Uploading file


In [9]:
from google.colab import files

# Upload video file
uploaded = files.upload()

# List uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


Saving intro.mp4 to intro.mp4
Uploaded file: intro.mp4


#2.  Converting file into readable format

In [11]:

import time

def upload_video(video_file_name):
  video_file = client.files.upload(path="intro.mp4")
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

pottery_video = upload_video('Pottery.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/8pk75yeaxwrz


#3. Giving video to gemini for scenes descriptions

In [37]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)
Scenes = response.text
Markdown(Scenes)

```json
[
  {
    "timecode": "00:00",
    "caption": "A young man with dark hair and a beard is speaking directly to the camera. He is wearing a light gray t-shirt."
  },
  {
    "timecode": "00:10",
   "caption": "The man continues to speak, articulating his words with an open mouth."
  },
{
    "timecode": "00:19",
   "caption": "The man speaks with conviction, looking directly into the camera, and showing his teeth when he speaks."
},
 {
    "timecode": "00:30",
   "caption": "The camera angle remains steady, with the focus on the man's face. He continues to speak with an open mouth, and he brings one hand into the frame."
  },
  {
    "timecode": "00:40",
    "caption": "The man's face fills the frame as he speaks with his eyes open."
  },
{
    "timecode": "00:50",
   "caption": "The lighting in the room is bright, as the man speaks, using hand gestures."
  },
  {
    "timecode": "00:57",
   "caption": "The man concludes the video with an open mouth and a direct gaze at the camera."
  }
]
```

In [24]:
!pip install gtts


#4 . Giving video to gemini for audio analysis

In [38]:
from gtts import gTTS
from google.genai.types import Content, Part


# Your prompt
prompt = """For each scene in this video,
            generate captions that describe the audio along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

# Generate response
response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
        prompt,
    ]
)

# Get the text response
output_text = response.text

# Print and speak the output
print(output_text)


```json
[
  {
    "timecode": "0:00",
    "caption": "A person begins to speak, \"Hello, my name is Mahmood Hussain.\""
  },
 {
    "timecode": "0:01",
    "caption": "The person continues, \"I'm a student with expertise in python, docker, get, github. I have 80% mastery in python including libraries like Numpy, Pandas.\""
  },
   {
    "timecode":"0:11",
    "caption": "The person continues, \"Additionally, I also know front end front end technologies including HTML, CSS, tailwind CSS, and Javascript.\""
  },
  {
    "timecode":"0:18",
   "caption":"The person continues, \"And I also have a strong grip on frameworks like NextJs, Django, and Typescript.\""
  },
  {
    "timecode":"0:23",
    "caption":"The person continues, \"I have completed several courses including artificial intelligence and machine learning, and prompt engineering and T3 stack which includes which focuses on typescript development.\""
   },
   {
    "timecode": "0:34",
    "caption":"The person continues, \"My han

#5.  Converting Gemini text responsw to audio using gTTS python library

In [34]:
# Function to play audio
def speak_output(message):
    tts = gTTS(text=message, lang='en')
    tts.save("response.mp3")
speak_output(output_text)


In [35]:
display(Audio("response.mp3", autoplay=True))

#6.Chatting with gemini on the basis on video

In [40]:

from google.genai.chats import Chat

# Initiate a chat
chat: Chat = client.chats.create(
    model=model
)

# Send chat a message
response = chat.send_message(output_text)
Markdown(response.text)

Okay, this is a good start for a transcript! Here are some ways we can analyze and potentially improve this data, depending on your goal:

**Analysis & Potential Improvements**

1.  **Spelling and Grammar:**
    *   "get" should be "Git" (assuming version control system).
    *   "front end front end" is redundant, should be just "front end".
    *   "My my" is a double word, should be "My"
    *   "Express and node js" should likely be "Express and Node.js".
    *   "UET at UET" sounds redundant, might be "UET" or "UET Lahore" or whatever is correct

2.  **Timecode Precision:** The timecodes are in seconds. If you're trying to sync with audio/video, you'd likely need millisecond accuracy (e.g., 0:00.345). If not, this is fine.

3.  **Clarity and Flow:**
    *   The phrase "My hands on project including" could be smoother. For example: "My hands-on projects include..." or "I have hands-on experience with projects like..."
    *   The last sentence is cut off. You need to know what he meant to say.

4.  **Contextual Understanding:**
    *   What does "BBIT" and "UET" stand for? Adding this information (or knowing that it's not relevant for the end goal) can be helpful.

5.  **Data Structure:** The JSON structure is straightforward and good for its purpose.

**Revised Transcript (with proposed improvements)**

```json
[
  {
    "timecode": "0:00",
    "caption": "Hello, my name is Mahmood Hussain."
  },
 {
    "timecode": "0:01",
    "caption": "I'm a student with expertise in Python, Docker, Git, and GitHub. I have 80% mastery in Python, including libraries like NumPy and Pandas."
  },
   {
    "timecode":"0:11",
    "caption": "Additionally, I also know front-end technologies including HTML, CSS, Tailwind CSS, and JavaScript."
  },
  {
    "timecode":"0:18",
   "caption":"And I also have a strong grip on frameworks like Next.js, Django, and TypeScript."
  },
  {
    "timecode":"0:23",
    "caption":"I have completed several courses including artificial intelligence and machine learning, and prompt engineering, and the T3 stack which focuses on TypeScript development."
   },
   {
    "timecode": "0:34",
    "caption":"My hands-on projects include creating a backend for a food app using Express and Node.js and an e-commerce website with Django and Tailwind CSS."
  },
   {
     "timecode":"0:43",
    "caption":"I also created a CV generator app using Python and Django."
   },
   {
    "timecode":"0:46",
    "caption": "Currently, I am studying at BBIT at UET and actively learning applied generative AI."
   },
   {
    "timecode": "0:54",
    "caption": "My teaching philosophy revolves around marketing..."
   }
]
```

**How to use this data:**

*   **Video Subtitles:** Use the timecodes and captions to generate subtitles for a video of Mahmood's introduction.
*   **Text Analysis:** Extract the skills and technologies mentioned for recruitment or analysis of their skillset.
*   **Presentation Outline:** Use the content to create a structured introduction for a presentation.
*   **Automated Transcript Enhancement:** This can serve as a base for NLP models to identify entities and suggest improvements.

**Next Steps:**

1.  **Listen to the Audio:** Carefully listen to the audio or watch the video to verify the accuracy of the timestamps and the actual words spoken, particularly the last unfinished sentence.
2.  **Confirm Terminology:** Check if "BBIT" and "UET" need clarification or expansion.
3.  **Define the Purpose:** Consider what you need the final output to be. This will shape how you polish the transcript.

Let me know if you have any more questions or further refinements needed.


In [41]:

response = chat.send_message("What is my person name in the video?")
Markdown(response.text)

Based on the provided transcript, your person's name is **Mahmood Hussain**.


In [42]:
response = chat.send_message("What is my person experties in the video?")
Markdown(response.text)

Based on the provided transcript, Mahmood Hussain's areas of expertise are:

*   **Programming Languages:** Python (with 80% mastery), JavaScript, TypeScript
*   **Libraries/Frameworks:** NumPy, Pandas, Next.js, Django, Tailwind CSS
*   **Tools:** Docker, Git, GitHub
*   **Concepts/Areas:** Artificial intelligence, machine learning, prompt engineering, applied generative AI, backend development, front-end development
*   **Platforms/Technologies:** Express, Node.js, HTML, CSS
